# Classification et explication des données

Nous allons maintenant utiliser les données pour faire de la classification et de l'explication. Nous allons utiliser les données de la base de données Image Net qui contient des images de différents animaux.

Pour réaliser cette tâhce, nous allons utiliser une ensemble d'image issue de la base de données Image Net. Nous allons utiliser les images de 6 classes différentes. Pour chaque classe, nous avons 99% images d'entrainement et 1% images de test. Les classes sont les suivantes :

- n02114367 (loup)
- n01484850 (requin)
- n01614925 (aigle)
- n02133161 (ours)
- n01537544 (passerin indigo)
- n01443537 (poisson rouge)

Nous allons utiliser les images d'entrainement pour entrainer un modèle de classification et les images de test pour évaluer la performance du modèle. Nous allons ensuite utiliser les images de test pour expliquer les prédictions du modèle.

Nous allons diviser notre travail différentes parties :

1) Récupérer les données sur Kaggle et sélectionner les images des classes qui nous intéressent (https://www.kaggle.com/c/imagenet-object-localization-challenge)
2) Traiter les données, cela implique de les diviser en classe égale et de les redimensionner, pour ce faire, nous pourrons utiliser des bibliothèques comme OpenCV ou Pillow
3) Diviser les données en données d'entrainement et données de test
4) Le choix du modèle de classification : nous utiliserons des modèles couramment utilisés pour la classification d'images incluent les réseaux de neurones convolutionnels (CNN), qui ont obtenu de bons résultats dans de nombreux domaines.
5) Entrainer un modèle de classification, pour ce faire, nous pourrons utiliser des bibliothèques comme Keras ou PyTorch ou TensorFlow
6) Évaluer la performance du modèle sur les données de test, et obtenir l'accuarcy du modèle
7) Utiliser les images de test pour expliquer les prédictions du modèle, pour ce faire, nous pourrons utiliser des bibliothèques comme LIME ou SHAP
